In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pandas as pd
import csv
import janitor
import numpy as np
import os
import glob

In [2]:
# read in original data

path = '/Users/danielle/Documents/thesis/paper-abundance-tables/profiles/*'                     
all_files = [name for name in glob.glob(path)]  

In [3]:
df_from_each_file = []
unclassified_list = []

for f in all_files:
    df = pd.read_csv(f, sep = '\t', skiprows=[0,1,2], usecols = ['#clade_name','relative_abundance']) # read in dataframe 
    
    id_1 = f.split('profiles/')[1] # add sample id from filename 
    id_2 = id_1.split('_S')[0]
    df["sampleid"] = id_2
    
    df.rename(columns = {'#clade_name':'taxa', 'relative_abundance':'abundance'}, inplace = True) 
    
    df = df[df['taxa'].str.contains("\|g__")] # keep genera
    df = df[~df['taxa'].str.contains("\|s__")] # keep species
    df["taxa"] = df['taxa'].str.split("\|g__").str[-1]
    df["taxa"] = df['taxa'].str.split("\|s__").str[0]
    
    # percent unclassified organisms profiled with mgx
    unclassified = sum(df.taxa.str.contains("_unclassified| noname | Candidatus"))/df.shape[0]
    unclassified_list.append(unclassified)
    
    # remove unclassified
    df = df[~df.taxa.str.contains("_unclassified")]
    df = df[~df.taxa.str.contains("_noname")]
    df = df[~df.taxa.str.contains("Candidatus")]
    
    # combine together taxa of the same genera
    df = df.groupby(['taxa', 'sampleid'])['abundance'].sum().reset_index()    
    
    # convert to relative abundance
    df["abundance"] = df["abundance"]/100.0
    
    df_from_each_file.append(df)

In [4]:
# averaging mean unclassified across all dataframes
np.mean(unclassified_list)*100

4.208356564349828

In [5]:
mgx = pd.concat(df_from_each_file, ignore_index=True) # concat all dataframes together

In [6]:
mgx.head()

,taxa,sampleid,abundance
0,Agathobaculum,C0047-7E-1A,0.001349
1,Alistipes,C0047-7E-1A,0.036088
2,Anaerostipes,C0047-7E-1A,0.010047
3,Asaccharobacter,C0047-7E-1A,0.000012
4,Bacteroides,C0047-7E-1A,0.652440


In [7]:
mgx_reshaped = mgx.pivot_table(index = "sampleid", values="abundance", columns = "taxa") # pivot

In [8]:
mgx_reshaped.columns.name = None

In [9]:
mgx_reshaped = mgx_reshaped.rename_axis(None, axis=1).reset_index()

In [10]:
mgx_reshaped = mgx_reshaped.fillna(0)  # fill in missing values with 0

In [11]:
mgx_reshaped

,sampleid,Absiella,Acetobacter,Acidaminococcus,Acinetobacter,Actinomyces,Actinotignum,Adlercreutzia,Aeriscardovia,Aeromonas,...,Stenotrophomonas,Streptococcus,Sutterella,Terrisporobacter,Turicibacter,Turicimonas,Tyzzerella,Varibaculum,Veillonella,Victivallis
0,C0047-7E-1A,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.000000,0.000156,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,C0052-7E-1A,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.001675,0.000000,0.0,0.002813,0.000000,0.000000,0.0,0.000000,0.0
2,C0053-8E-1A,0.0,0.0,0.0,0.0,0.001754,0.0,0.000623,0.000000,0.0,...,0.0,0.008166,0.000000,0.0,0.000670,0.000062,0.001062,0.0,0.000000,0.0
3,C0058-4E-1A,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.003549,0.000000,0.0,0.002141,0.000000,0.000000,0.0,0.001234,0.0
4,C0059-4E-1A,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.005601,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,C1177-4F-1A,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000079,0.0,...,0.0,0.009370,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
126,C1219-3F-1A,0.0,0.0,0.0,0.0,0.000153,0.0,0.000000,0.000000,0.0,...,0.0,0.032457,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.025505,0.0
127,C2002-3E-1A,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,...,0.0,0.093448,0.000000,0.0,0.001969,0.000000,0.000182,0.0,0.037591,0.0
128,C2014-2E-1A,0.0,0.0,0.0,0.0,0.002798,0.0,0.000000,0.000000,0.0,...,0.0,0.001256,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.013814,0.0


In [12]:
# adding age metadata
age = pd.read_csv("/Users/danielle/Documents/thesis/paper-abundance-tables/metadata/joined.csv")

In [13]:
age.head()

,sample,subject,timepoint,batch,childGender,correctedAgeDays,mother_HHS,childBMI,hasScan,mcDespotProcessed,...,rt_Pallidum,rt_Hippocampus,rt_Amygdala,rt_Insula,rt_Operculum,hires_total,cerebellar,neocortical,limbic,subcortical
0,C0052_5F_1A,52,5,1,Male,2824.0,6.0,16.728348,1.0,1.0,...,1315.875122,2491.843506,1107.906250,536.937500,570.96875,264479.536024,89286.656250,127223.941542,34246.781738,47968.938232
1,C0126_8F_1A,126,8,1,Male,2261.0,7.0,NaN,1.0,0.0,...,1486.031250,2737.625000,1210.000122,680.625000,616.34375,320037.446868,100407.316406,163357.565277,41079.502685,56272.565185
2,C0126_8F_1B,126,8,1,Male,2261.0,7.0,NaN,1.0,0.0,...,1486.031250,2737.625000,1210.000122,680.625000,616.34375,320037.446868,100407.316406,163357.565277,41079.502685,56272.565185
3,C0127_6F_1B,127,6,1,Female,3367.0,7.0,NaN,1.0,1.0,...,1338.562500,2230.937500,1089.000000,654.156311,567.18750,270177.880217,86435.597656,135330.938994,34012.343781,48411.343567
4,C0286_5F_1A,286,5,1,Male,1926.0,6.0,NaN,1.0,0.0,...,1603.250000,2457.812500,1070.093750,616.343750,548.28125,284418.066527,88447.218750,143566.502440,36700.813782,52404.345337


In [14]:
# change characters in sample names 
age["sample"] = age["sample"].str.replace("_",'-')

In [15]:
age["AgeMonths"] = age["correctedAgeDays"]/30.0
age.rename(columns = {'sample':'sampleid'}, inplace = True) 

In [16]:
# make age dictionary
agedict = {str(s): {} for s in age["sampleid"]}
for index, row in age.iterrows():
    age_months = row["AgeMonths"]
    agedict[row["sampleid"]]= age_months

In [17]:
mgx_reshaped["AgeMonths"]= mgx_reshaped["sampleid"].map(agedict)

In [18]:
def assign_dev_stage(x):
    if x<15:
        return "less than 15 months"
    elif 15 <=x<= 30:
        return "15 to 30 months"
    elif x>30:
        return "older than 30 months"

In [19]:
mgx_reshaped['dev_stage'] = mgx_reshaped['AgeMonths'].apply(assign_dev_stage)

In [20]:
# re-order columns

cols_to_order = ['sampleid','AgeMonths', 'dev_stage']
new_columns = cols_to_order + (mgx_reshaped.columns.drop(cols_to_order).tolist())
mgx_reshaped = mgx_reshaped[new_columns]

In [21]:
mgx_reshaped

,sampleid,AgeMonths,dev_stage,Absiella,Acetobacter,Acidaminococcus,Acinetobacter,Actinomyces,Actinotignum,Adlercreutzia,...,Stenotrophomonas,Streptococcus,Sutterella,Terrisporobacter,Turicibacter,Turicimonas,Tyzzerella,Varibaculum,Veillonella,Victivallis
0,C0047-7E-1A,97.233333,older than 30 months,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.000156,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
1,C0052-7E-1A,127.333333,older than 30 months,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.001675,0.000000,0.0,0.002813,0.000000,0.000000,0.0,0.000000,0.0
2,C0053-8E-1A,105.700000,older than 30 months,0.0,0.0,0.0,0.0,0.001754,0.0,0.000623,...,0.0,0.008166,0.000000,0.0,0.000670,0.000062,0.001062,0.0,0.000000,0.0
3,C0058-4E-1A,137.766667,older than 30 months,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.003549,0.000000,0.0,0.002141,0.000000,0.000000,0.0,0.001234,0.0
4,C0059-4E-1A,137.766667,older than 30 months,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.005601,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,C1177-4F-1A,1.666667,less than 15 months,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.009370,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0
126,C1219-3F-1A,1.266667,less than 15 months,0.0,0.0,0.0,0.0,0.000153,0.0,0.000000,...,0.0,0.032457,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.025505,0.0
127,C2002-3E-1A,8.600000,less than 15 months,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,...,0.0,0.093448,0.000000,0.0,0.001969,0.000000,0.000182,0.0,0.037591,0.0
128,C2014-2E-1A,6.100000,less than 15 months,0.0,0.0,0.0,0.0,0.002798,0.0,0.000000,...,0.0,0.001256,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.013814,0.0


In [22]:
mgx_reshaped.to_csv('mgx_abund_df.csv', index=False)

In [23]:
# read in 16S data
amplicon = pd.read_csv("/Users/danielle/Documents/thesis/paper-abundance-tables/dada2_output_exported/feature-table.txt",
                      skiprows=[0], sep='\t')

In [24]:
amplicon.drop(['#OTU ID'], axis = 1, inplace = True)
amplicon.rename(columns = {"taxonomy": "taxa"}, inplace = True)

In [25]:
cols = list(amplicon.columns)
cols = [cols[-1]] + cols[:-1]
amplicon = amplicon[cols]

In [26]:
# clean taxonomic names
# keep only genera
amplicon["taxa"] = amplicon['taxa'].str.split("D_5__", expand=True)[1]

In [27]:
amplicon["taxa"] = amplicon['taxa'].str.split(";", expand=True)[0]

In [28]:
amplicon["taxa"] = amplicon['taxa'].str.split("_", expand=True)[0]

In [29]:
amplicon["taxa"] = amplicon['taxa'].str.split("-", expand=True)[0]

In [30]:
amplicon["taxa"] = amplicon["taxa"].str.strip('[]').astype(str)

In [31]:
amplicon.head()

,taxa,C0047-7E-1A,C0052-7E-1A,C0053-8E-1A,C0058-4E-1A,C0059-4E-1A,C0106-6E-1A,C0107-4E-1A,C0123-3E-1A,C0126-9E-1A,...,C1102-1F-1A,C1109-4F-1A,C1115-4F-1A,C1135-4F-1A,C1160-3F-1A,C1177-4F-1A,C1219-3F-1A,C2002-3E-1A,C2014-2E-1A,C2018-3E-1A
0,Bacteroides,0.0,1163.0,0.0,0.0,0.0,0.0,0.0,214.0,1476.0,...,369.0,0.0,0.0,0.0,9.0,336.0,0.0,577.0,0.0,376.0
1,Escherichia,0.0,5.0,42.0,0.0,0.0,7.0,0.0,0.0,0.0,...,4.0,0.0,637.0,593.0,1559.0,0.0,14633.0,18.0,1574.0,13.0
2,Prevotella,0.0,1595.0,0.0,0.0,0.0,0.0,7.0,0.0,0.0,...,826.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8823.0
3,Bacteroides,0.0,114.0,0.0,0.0,0.0,0.0,0.0,79.0,533.0,...,420.0,19.0,0.0,32.0,0.0,0.0,1801.0,889.0,0.0,0.0
4,Bacteroides,1289.0,0.0,0.0,2493.0,5670.0,4056.0,1143.0,0.0,0.0,...,831.0,0.0,0.0,0.0,0.0,13995.0,0.0,198.0,0.0,0.0


In [32]:
# combine together taxa of the same genera
amplicon = amplicon.groupby(['taxa']).sum().reset_index()

In [33]:
amplicon.sort_values(by = "taxa", ascending=True, inplace=True)

In [34]:
# calculate relative abundances
bug_sum = amplicon.sum(axis = 0, skipna = True)
amplicon.iloc[:, 1:] = amplicon.iloc[:, 1:]/bug_sum

In [35]:
amplicon_melt = pd.melt(amplicon, id_vars=["taxa"], var_name = "sampleid", value_name = "amplicon_abund")

In [36]:
# remove if genus is less than 5 characters
remove = list(set([name for name in amplicon_melt["taxa"] if len(name)<5]))

In [37]:
# remove if numbers in genus
remove2 = list(set([name for name in amplicon_melt["taxa"] if not name.isalpha()]))

In [38]:
# remove if Candidatus in genus
remove3 = list(set([name for name in amplicon_melt["taxa"] if "Candidatus" in name]))

In [39]:
remove_list = remove + remove2 + remove3 + ["uncultured", "Family", "None", "Clade", "Subgroup", "Ruminococcaceae"]

In [40]:
# percent unclassified organisms profiled with 16S
np.sum([amplicon_melt.taxa.isin(remove_list)])/(amplicon_melt["taxa"].shape[0])*100

9.239130434782608

In [41]:
amplicon_melt = amplicon_melt[~amplicon_melt.taxa.isin(remove_list)]

In [42]:
amplicon_reshaped = amplicon_melt.pivot_table(index = "sampleid", values="amplicon_abund", columns = "taxa") # pivot

In [43]:
amplicon_reshaped.columns.name = None

In [44]:
amplicon_reshaped = amplicon_reshaped.rename_axis(None, axis=1).reset_index()

In [45]:
amplicon_reshaped = amplicon_reshaped.fillna(0)  # fill in missing values with 0

In [46]:
amplicon_reshaped = amplicon_reshaped.loc[:,~amplicon_reshaped.columns.str.contains(r'[0-9]', case=False)] 

In [47]:
amplicon_reshaped

,sampleid,Acetanaerobacterium,Achromobacter,Acidaminococcus,Acinetobacter,Actinobacillus,Actinomyces,Adlercreutzia,Aeromonas,Agathobacter,...,Sulfitobacter,Sutterella,Terrisporobacter,Turicibacter,Tyzzerella,Ulvibacter,Varibaculum,Veillonella,Victivallis,Virgulinella
0,C0047-7E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.022783,...,0.00000,0.018552,0.000000,0.000271,0.000000,0.0,0.0,0.000000,0.0,0.0
1,C0052-7E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.013665,...,0.00000,0.006443,0.005048,0.008125,0.000000,0.0,0.0,0.000657,0.0,0.0
2,C0053-8E-1A,0.000000,0.0,0.0,0.0,0.0,0.000678,0.000000,0.0,0.005422,...,0.00000,0.000000,0.000000,0.001355,0.030497,0.0,0.0,0.000387,0.0,0.0
3,C0058-4E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000273,0.0,0.011691,...,0.00000,0.000000,0.002065,0.006937,0.000000,0.0,0.0,0.002650,0.0,0.0
4,C0059-4E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000403,0.0,0.101117,...,0.00000,0.000000,0.004730,0.002290,0.000000,0.0,0.0,0.001359,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,C1177-4F-1A,0.000157,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.00000,0.013042,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
126,C1219-3F-1A,0.000000,0.0,0.0,0.0,0.0,0.000028,0.000000,0.0,0.000000,...,0.00000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.060177,0.0,0.0
127,C2002-3E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.00000,0.000000,0.031022,0.005005,0.016784,0.0,0.0,0.017604,0.0,0.0
128,C2014-2E-1A,0.000000,0.0,0.0,0.0,0.0,0.000480,0.000000,0.0,0.000000,...,0.00012,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.026844,0.0,0.0


In [48]:
amplicon_reshaped.to_csv('16S_abundance.csv')

In [49]:
chlorobium = amplicon_reshaped["Chlorobium"]

In [50]:
np.mean(chlorobium[chlorobium!= 0])

0.0004351203788975447

### looking for the intersection of taxonomy

In [51]:
amplicon_taxa_list = set(amplicon_reshaped.columns.values)
mgx_taxa_list = set(mgx_reshaped.columns.values)

In [52]:
amplicon_taxa_list

{'Acetanaerobacterium',
 'Achromobacter',
 'Acidaminococcus',
 'Acinetobacter',
 'Actinobacillus',
 'Actinomyces',
 'Adlercreutzia',
 'Aeromonas',
 'Agathobacter',
 'Akkermansia',
 'Alistipes',
 'Allisonella',
 'Amylibacter',
 'Anaerococcus',
 'Anaerofustis',
 'Anaerosporobacter',
 'Anaerostipes',
 'Anaerotruncus',
 'Angelakisella',
 'Arenicella',
 'Atopobium',
 'Azospirillum',
 'Bacillus',
 'Bacteroides',
 'Barnesiella',
 'Bifidobacterium',
 'Bilophila',
 'Blastopirellula',
 'Blautia',
 'Brevundimonas',
 'Butyricicoccus',
 'Butyricimonas',
 'Campylobacter',
 'Caproiciproducens',
 'Catenibacterium',
 'Chlorobium',
 'Christensenellaceae',
 'Chryseobacterium',
 'Chrysosporum',
 'Clostridioides',
 'Clostridium',
 'Collinsella',
 'Comamonas',
 'Coprobacillus',
 'Coprobacter',
 'Coprococcus',
 'Corynebacterium',
 'Crocinitomix',
 'Cutibacterium',
 'Defluviitaleaceae',
 'Desulfovibrio',
 'Dialister',
 'Dielma',
 'Dorea',
 'Dysgonomonas',
 'Eggerthella',
 'Eisenbergiella',
 'Enhydrobacter',
 

In [53]:
mgx_taxa_list

{'Absiella',
 'Acetobacter',
 'Acidaminococcus',
 'Acinetobacter',
 'Actinomyces',
 'Actinotignum',
 'Adlercreutzia',
 'Aeriscardovia',
 'Aeromonas',
 'Agathobaculum',
 'AgeMonths',
 'Akkermansia',
 'Alistipes',
 'Allisonella',
 'Anaerococcus',
 'Anaerofustis',
 'Anaeromassilibacillus',
 'Anaerostipes',
 'Anaerotignum',
 'Anaerotruncus',
 'Asaccharobacter',
 'Atlantibacter',
 'Atopobium',
 'Bacteroides',
 'Barnesiella',
 'Bifidobacterium',
 'Bilophila',
 'Blautia',
 'Butyricicoccus',
 'Butyricimonas',
 'Campylobacter',
 'Catenibacterium',
 'Chryseobacterium',
 'Citrobacter',
 'Clostridioides',
 'Clostridium',
 'Collinsella',
 'Comamonas',
 'Coprobacillus',
 'Coprobacter',
 'Coprococcus',
 'Corynebacterium',
 'Desulfovibrio',
 'Dialister',
 'Dielma',
 'Dorea',
 'Eggerthella',
 'Eikenella',
 'Eisenbergiella',
 'Elizabethkingia',
 'Enorma',
 'Enterobacter',
 'Enterococcus',
 'Erysipelatoclostridium',
 'Escherichia',
 'Eubacterium',
 'Faecalibacterium',
 'Faecalicatena',
 'Faecalicoccus',


In [54]:
exclude = ["sampleid", "dev_stage", "AgeMonths"]

In [55]:
intersection = [taxa for taxa in amplicon_taxa_list.intersection(mgx_taxa_list) if taxa not in exclude]
union = [taxa for taxa in amplicon_taxa_list.union(mgx_taxa_list) if taxa not in exclude]

In [56]:
len(amplicon_taxa_list) + len(mgx_taxa_list)

310

In [57]:
amp_only = [taxa for taxa in amplicon_taxa_list.difference(intersection) if taxa not in exclude]
mgx_only = [taxa for taxa in mgx_taxa_list.difference(intersection) if taxa not in exclude]

In [58]:
len(intersection) # genera found in both
len(amp_only) # genera found in amplicon only
len(mgx_only) # genera found in mgx only

105

62

34

In [59]:
with open("unique_amplicon.txt", "w") as output:
    output.write(str(amp_only))

1049

In [60]:
with open("unique_mgx.txt", "w") as output:
    output.write(str(mgx_only))

557

In [61]:
with open("union_genera.txt", "w") as output:
    output.write(str(union))

3252

In [62]:
with open('intersection.txt', mode='wt', encoding='utf-8') as myfile:
    myfile.write('\n'.join(intersection))

1330

In [63]:
database_info = pd.read_csv("union_taxa.csv")

In [64]:
database_info.head()

,taxa,in_opp_db,method,few_seqs,Unnamed: 4
0,Erysipelotrichaceae,NaN,amp,NaN,NaN
1,Sphingobacterium,NaN,amp,NaN,NaN
2,Mogibacterium,NaN,amp,NaN,NaN
3,Lysinibacillus,NaN,amp,NaN,NaN
4,Planktotalea,NaN,amp,NaN,NaN


In [65]:
database_dict = database_info.set_index('taxa').to_dict('dict')

In [66]:
database_dict = database_dict['in_opp_db']

In [67]:
# read in species list used to make tree with branch lengths
tree_species = pd.read_csv("/Users/danielle/Documents/thesis/paper-phylogeny/species_timetree.txt", header=None)

In [68]:
tree_species.columns = ['species']

In [69]:
tree_species['species'] = tree_species['species'].str.replace(' ', '_')


In [70]:
tree_species["genus"] = tree_species['species'].str.split("_").str[0]

In [71]:
 tree_species.head()

,species,genus
0,Methanobrevibacter_curvatus,Methanobrevibacter
1,Rhodopirellula_rosea,Rhodopirellula
2,Blastopirellula_marina,Blastopirellula
3,Lewinella_persica,Lewinella
4,Sphingobacterium_cladoniae,Sphingobacterium


In [72]:
tree_species["amp"] = [1 if x in amplicon_taxa_list else -1 for x in tree_species["genus"]]

In [73]:
tree_species["mgx"] = [1 if x in mgx_taxa_list  else -1 for x in tree_species["genus"]]

In [74]:
tree_species["in_opp_db"] = tree_species["genus"].map(database_dict)
tree_species.fillna(-1, inplace = True)  # fill in missing values with 0

In [75]:
tree_species.drop(columns=['species'], inplace=True)

In [76]:
tree_species.sample(5)

,genus,amp,mgx,in_opp_db
59,Azospirillum,1,-1,-1.0
76,Megasphaera,1,1,-1.0
22,Flavobacterium,1,-1,-1.0
88,Lactonifactor,-1,1,-1.0
92,Faecalibacterium,1,1,-1.0


In [77]:
pd.DataFrame(tree_species).to_csv('/Users/danielle/Documents/thesis/paper-phylogeny/phylo_tree_labels.txt', 
                                   index=False, header=False)

In [78]:
pd.DataFrame(union).to_csv('/Users/danielle/Documents/thesis/paper-phylogeny/all_taxa.txt', index=False, header=False)

### combine two dataframes, long form


In [79]:
mgx = mgx.fillna(0)  # fill in missing values with 0

In [80]:
merged_taxa = pd.merge(amplicon_melt, mgx, on = ["sampleid", "taxa"], how = "outer").fillna(0)
merged_taxa["mgx_abund"] = merged_taxa["abundance"]

In [81]:
merged_taxa["abs_diff"] = abs(merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])
merged_taxa["tot_diff"] = (merged_taxa["amplicon_abund"] - merged_taxa["mgx_abund"])

In [82]:
merged_taxa["AgeMonths"]= merged_taxa["sampleid"].map(agedict)
merged_taxa['dev_stage'] = merged_taxa['AgeMonths'].apply(assign_dev_stage)

In [83]:
# re-order columns

cols_to_order = ['sampleid','AgeMonths', 'dev_stage']
new_columns = cols_to_order + (merged_taxa.columns.drop(cols_to_order).tolist())
merged_taxa = merged_taxa[new_columns]

In [84]:
merged_taxa

,sampleid,AgeMonths,dev_stage,taxa,amplicon_abund,abundance,mgx_abund,abs_diff,tot_diff
0,C0047-7E-1A,97.233333,older than 30 months,Acetanaerobacterium,0.0,0.000000,0.000000,0.000000,0.000000
1,C0047-7E-1A,97.233333,older than 30 months,Achromobacter,0.0,0.000000,0.000000,0.000000,0.000000
2,C0047-7E-1A,97.233333,older than 30 months,Acidaminococcus,0.0,0.000000,0.000000,0.000000,0.000000
3,C0047-7E-1A,97.233333,older than 30 months,Acinetobacter,0.0,0.000000,0.000000,0.000000,0.000000
4,C0047-7E-1A,97.233333,older than 30 months,Actinobacillus,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
22075,C1177-4F-1A,1.666667,less than 15 months,Aeriscardovia,0.0,0.000079,0.000079,0.000079,-0.000079
22076,C1177-4F-1A,1.666667,less than 15 months,Gordonibacter,0.0,0.000029,0.000029,0.000029,-0.000029
22077,C1177-4F-1A,1.666667,less than 15 months,Ruthenibacterium,0.0,0.004162,0.004162,0.004162,-0.004162
22078,C2018-3E-1A,12.100000,less than 15 months,Agathobaculum,0.0,0.002942,0.002942,0.002942,-0.002942


In [85]:
merged_taxa.to_csv('paper_abund_df.csv', index=False)

## making giant dataframe of abundances

In [86]:
amplicon_reshaped["method"] = "amp"
mgx_reshaped["method"] = "mgx"

/Users/danielle/miniconda2/envs/thesis_analysis/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [87]:
amplicon_reshaped["AgeMonths"]= amplicon_reshaped["sampleid"].map(agedict)
amplicon_reshaped['dev_stage'] = amplicon_reshaped['AgeMonths'].apply(assign_dev_stage)

In [88]:
amplicon_reshaped

,sampleid,Acetanaerobacterium,Achromobacter,Acidaminococcus,Acinetobacter,Actinobacillus,Actinomyces,Adlercreutzia,Aeromonas,Agathobacter,...,Turicibacter,Tyzzerella,Ulvibacter,Varibaculum,Veillonella,Victivallis,Virgulinella,method,AgeMonths,dev_stage
0,C0047-7E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.022783,...,0.000271,0.000000,0.0,0.0,0.000000,0.0,0.0,amp,97.233333,older than 30 months
1,C0052-7E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.013665,...,0.008125,0.000000,0.0,0.0,0.000657,0.0,0.0,amp,127.333333,older than 30 months
2,C0053-8E-1A,0.000000,0.0,0.0,0.0,0.0,0.000678,0.000000,0.0,0.005422,...,0.001355,0.030497,0.0,0.0,0.000387,0.0,0.0,amp,105.700000,older than 30 months
3,C0058-4E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000273,0.0,0.011691,...,0.006937,0.000000,0.0,0.0,0.002650,0.0,0.0,amp,137.766667,older than 30 months
4,C0059-4E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000403,0.0,0.101117,...,0.002290,0.000000,0.0,0.0,0.001359,0.0,0.0,amp,137.766667,older than 30 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,C1177-4F-1A,0.000157,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,amp,1.666667,less than 15 months
126,C1219-3F-1A,0.000000,0.0,0.0,0.0,0.0,0.000028,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.060177,0.0,0.0,amp,1.266667,less than 15 months
127,C2002-3E-1A,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.005005,0.016784,0.0,0.0,0.017604,0.0,0.0,amp,8.600000,less than 15 months
128,C2014-2E-1A,0.000000,0.0,0.0,0.0,0.0,0.000480,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.026844,0.0,0.0,amp,6.100000,less than 15 months


In [89]:
concat_df = pd.concat([mgx_reshaped, amplicon_reshaped], sort=True).reset_index(drop = True)
concat_df = concat_df.fillna(0)

In [90]:
# re-order columns

cols_to_order = ['sampleid','AgeMonths', 'dev_stage', "method"]
new_columns = cols_to_order + (concat_df.columns.drop(cols_to_order).tolist())
concat_df = concat_df[new_columns]

In [91]:
concat_df.sample(15)

,sampleid,AgeMonths,dev_stage,method,Absiella,Acetanaerobacterium,Acetobacter,Achromobacter,Acidaminococcus,Acinetobacter,...,Sutterella,Terrisporobacter,Turicibacter,Turicimonas,Tyzzerella,Ulvibacter,Varibaculum,Veillonella,Victivallis,Virgulinella
42,C0674-3F-1A,3.433333,less than 15 months,mgx,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
257,C2002-3E-1A,8.600000,less than 15 months,amp,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.031022,0.005005,0.000000,0.016784,0.0,0.0,0.017604,0.0,0.0
140,C0175-2F-1A,131.100000,older than 30 months,amp,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.001630,0.001293,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0
248,C1090-1F-1A,3.800000,less than 15 months,amp,0.0,0.0,0.0,0.0,0.0,0.000057,...,0.014205,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.001130,0.0,0.0
36,C0654-2F-1A,18.700000,15 to 30 months,mgx,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000508,0.000000,0.0,0.0,0.003331,0.0,0.0
137,C0123-3E-1A,93.066667,older than 30 months,amp,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.003067,0.0,0.0,0.025153,0.0,0.0
188,C0725-1E-1A,26.800000,15 to 30 months,amp,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.021108,0.015057,0.012127,0.000000,0.001024,0.0,0.0,0.001215,0.0,0.0
121,C1109-4F-1A,3.433333,less than 15 months,mgx,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.106267,0.0,0.0
25,C0494-2E-1A,27.333333,15 to 30 months,mgx,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.000000,0.002058,0.000298,0.000000,0.0,0.0,0.000000,0.0,0.0
218,C0886-4F-1A,5.666667,less than 15 months,amp,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.009182,0.000000,0.000000,0.000000,0.005430,0.0,0.0,0.004768,0.0,0.0


In [92]:
concat_df.to_csv('transposed_mgxamp_df.csv',index=False)